In [ ]:
import os

from torchviz import make_dot
import torch
os.chdir("./recognition/arcface_torch")

In [ ]:

from partial_fc import PartialFC, PartialFCAdamW
from backbones import get_model
import logging
from utils.utils_config import get_config
from losses import CombinedMarginLoss
cfg=get_config(r"configs/glint360k_r50")

margin_loss = CombinedMarginLoss(
    64,
    cfg.margin_list[0],
    cfg.margin_list[1],
    cfg.margin_list[2],
    cfg.interclass_filtering_threshold
)

backbone = get_model(
    cfg.network, dropout=0.0, fp16=cfg.fp16, num_features=cfg.embedding_size).cuda()
backbone.eval()
torch.cuda.set_device(0)

In [ ]:

x = (torch.rand(1,3,112,112)-0.5)/0.5
x=x.cuda()
y = backbone(x)

make_dot(y.mean(), params=dict(backbone.named_parameters()))

In [ ]:
module_partial_fc = PartialFC(
    margin_loss, cfg.embedding_size, cfg.num_classes,
    cfg.sample_rate, cfg.fp16)